In [1]:
# Requires brainvisa

In [2]:
import anatomist.api as anatomist
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

In [3]:
from soma import aims
import json
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np
import glob

In [4]:
%matplotlib qt5
a = anatomist.Anatomist()

Starting Anatomist.....
global modules: /neurospin/dico/rmenasria/Runs/03_main/Program/main_pixi/.pixi/envs/default/share/anatomist-5.2/python_plugins
home   modules: /home/rm283713/.anatomist/python_plugins
loading module gltf_io
config file : /home/rm283713/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module palettecontrols
loading module paletteViewer
loading module meshsplit
loading module profilewindow
loading module ana_image_math
loading module anacontrolmenu
loading module foldsplit
loading module modelGraphs
loading module bsa_proba
loading module histogram
loading module gradientpalette
loading module infowindow
loading module simple_controls
loading module volumepalettes
loading module statsplotwindow
loading module save_resampled
loading module valuesplotwindow
loading module selection
all python modules loaded
Anatomist started.


In [5]:
Rspam_model = "/neurospin/dico/rmenasria/Runs/01_essai/Program/env_pixi/.pixi/envs/default/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

Lspam_model = "/neurospin/dico/rmenasria/Runs/01_essai/Program/env_pixi/.pixi/envs/default/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_left/meshes/Lspam_model_meshes_1.arg"


json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_gridsearch.json"

path_summary = ""
path_file = "/neurospin/dico/rmenasria/Runs/03_main/Output/csv/all_sulci_gravity_center_shifts_utrecht_pclean.csv"


In [6]:
with open(json_regions) as f:
    regions = json.load(f)
    print(regions)

{'brain': {'S.C.-sylv._left': {'S.C._left': ['S.C._left'], 'S.C.sylvian._left': ['S.C.sylvian._left']}, 'S.C.-S.Pe.C._left': {'S.C._left': ['S.C._left'], 'S.C.sylvian._left': ['S.C.sylvian._left'], 'S.Pe.C.inf._left': ['S.Pe.C.inf._left'], 'S.Pe.C.inter._left': ['S.Pe.C.inter._left'], 'S.Pe.C.marginal._left': ['S.Pe.C.marginal._left'], 'S.Pe.C.sup._left': ['S.Pe.C.sup._left'], 'S.Pe.C.median._left': ['S.Pe.C.median._left']}, 'S.C.-S.Po.C._left': {'S.C._left': ['S.C._left'], 'S.C.sylvian._left': ['S.C.sylvian._left'], 'S.Po.C.sup._left': ['S.Po.C.sup._left'], 'F.I.P.Po.C.inf._left': ['F.I.P.Po.C.inf._left'], 'F.C.L.r.retroC.tr._left': ['F.C.L.r.retroC.tr._left']}, 'S.Pe.C._left': {'S.Pe.C.inf._left': ['S.Pe.C.inf._left'], 'S.Pe.C.inter._left': ['S.Pe.C.inter._left'], 'S.Pe.C.marginal._left': ['S.Pe.C.marginal._left'], 'S.Pe.C.sup._left': ['S.Pe.C.sup._left'], 'S.Pe.C.median._left': ['S.Pe.C.median._left']}, 'S.Po.C._left': {'S.Po.C.sup._left': ['S.Po.C.sup._left'], 'F.I.P.Po.C.inf._left

In [7]:
file_to_display = f"{path_summary}/{path_file}"

In [8]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [9]:
param = "shift_z"

In [10]:
def normalize_sulcus_name(name):
    """
    Applies normalization to the sulcus name.
    This function replaces specific substrings in the sulcus name
    with their corresponding normalized versions.
    """
    name = name.replace("CINGULATE", "CINGULATE.")
    name = name.replace("ORBITAL", "S.Or.")
    name = name.replace("paracingular.", "S.F.int.")
    return name

In [11]:
df = pd.read_csv(path_file)  
# Add a side column
df['side'] = df['sulcus'].str.endswith("_right").map({True:"R", False:"L"})
# Create a new column 'sulcus'respecting the naming convention
df['vertex_name'] = df['sulcus'].map(normalize_sulcus_name)



In [12]:
all_dict_sulci = set()
for macro in regions['brain'].values():
    all_dict_sulci.update(macro.keys())

missing = [v for v in df['vertex_name'] if v not in all_dict_sulci]
if missing:
    print("The following names will not match :", missing)
else:
    print("All names are present in the dictionary")

df = df[~df['vertex_name'].isin(missing)]

print(df.head())

The following names will not match : ['S.Li._right', 'S.Li._left']
                    sulcus   shift_x   shift_y   shift_z   distance  \
0  F.Cal.ant.-Sc.Cal._left  0.025461  5.364980  0.751552   5.417425   
1      F.I.P.r.int.1_right  1.051453  7.599596 -7.450844  10.694601   
2      F.C.L.r.diag._right -5.585739  6.422802 -4.592613   9.671864   
3           S.F.inf._right -4.798425  7.973393 -4.764624  10.454737   
4       F.I.P.r.int.1_left  4.234046  7.651918  0.463159   8.757483   

   nbr_preterms  nbr_adults  std_preterms_x  std_preterms_y  std_preterms_z  \
0            71          62        4.194018        5.115897        5.408297   
1            65          51        4.705701        7.333243        8.889529   
2            69          31        2.391627        5.439522        6.392049   
3            71          62        3.096929        5.771577        5.942055   
4            64          57        4.365501        7.155567        7.143954   

   std_adults_x  std_adults_y  

In [13]:
dic = {
    "FCLp-subsc-FCLa-INSULA": "F.C.L.p.-subsc.-F.C.L.a.-INSULA.",
    "FCMpost-SpC": "F.C.M.post.-S.p.C.",
    "FColl-SRh": "F.Coll.-S.Rh.",
    "FIP": "F.I.P.",
    "FPO-SCu-ScCal": "F.P.O.-S.Cu.-Sc.Cal.",
    "Lobule_parietal_sup": "Lobule_parietal_sup.",
    "SOr": "S.Or.",
    "SOlf": "S.Olf.",
    "SC-SPeC": "S.C.-S.Pe.C.",
    "SC-SPoC": "S.C.-S.Po.C.",
    "SC-sylv": "S.C.-sylv.",
    "SFinf-BROCA-SPeCinf": "S.F.inf.-BROCA-S.Pe.C.inf.",
    "SFint-FCMant": "S.F.int.-F.C.M.ant.",
    "SFint-SR": "S.F.int.-S.R.",
    "SFinter-SFsup": "S.F.inter.-S.F.sup.",
    "SFmarginal-SFinfant":"S.F.marginal-S.F.inf.ant.",
    "SFmedian-SFpoltr-SFsup": "S.F.median-S.F.pol.tr.-S.F.sup.",
    "SOr-SOlf": "S.Or.-S.Olf.",
    "SPeC": "S.Pe.C.",
    "SPoC": "S.Po.C.",
    "STi-SOTlat": "S.T.i.-S.O.T.lat.",
    "STi-STs-STpol": "S.T.i.-S.T.s.-S.T.pol.",
    "STsbr": "S.T.s.br.",
    "STs": "S.T.s.",
    "ScCal-SLi": "Sc.Cal.-S.Li.",
    "SsP-SPaint": "S.s.P.-S.Pa.int.",
}

In [14]:
for key in dic.keys():
    df["vertex_name"] = df["vertex_name"].str.replace(key, dic[key])
df.head()

,sulcus,shift_x,shift_y,shift_z,distance,nbr_preterms,nbr_adults,std_preterms_x,std_preterms_y,std_preterms_z,std_adults_x,std_adults_y,std_adults_z,side,vertex_name
0,F.Cal.ant.-Sc.Cal._left,0.025461,5.364980,0.751552,5.417425,71,62,4.194018,5.115897,5.408297,1.749477,3.974745,3.013375,L,F.Cal.ant.-Sc.Cal._left
1,F.I.P.r.int.1_right,1.051453,7.599596,-7.450844,10.694601,65,51,4.705701,7.333243,8.889529,3.409731,5.695065,8.961436,R,F.I.P.r.int.1_right
2,F.C.L.r.diag._right,-5.585739,6.422802,-4.592613,9.671864,69,31,2.391627,5.439522,6.392049,1.810037,3.390500,4.311721,R,F.C.L.r.diag._right
3,S.F.inf._right,-4.798425,7.973393,-4.764624,10.454737,71,62,3.096929,5.771577,5.942055,1.864670,4.092421,3.844847,R,S.F.inf._right
4,F.I.P.r.int.1_left,4.234046,7.651918,0.463159,8.757483,64,57,4.365501,7.155567,7.143954,3.741433,5.278325,5.922729,L,F.I.P.r.int.1_left


In [15]:
def build_gradient(pal):
    """Builds a gradient palette."""
    gw = anatomist.cpp.GradientWidget(
        None, 'gradientwidget',
        pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0,
                                :3][:, ::-1]  # Convert BGRA to RGBA
    pal.update()

In [16]:
# Split the dataframe into two dataframes based on the side
df_left = df[df['side'] == 'L']
df_right = df[df['side'] == 'R']

In [17]:
min_shift = None
max_shift = None




def set_color_property(res, side):
    global dic
    global dic, min_shift, max_shift 

    min_shift = res[param].min()
    max_shift = res[param].max()
    print(f"min_shift: {min_shift}, max_shift: {max_shift}")

    if side == "L":
        spam_model_file = Lspam_model
    else:
        spam_model_file = Rspam_model
        
    dic[f"aims{side}"] = aims.read(spam_model_file)

    vertex_index = {}
    for vertex in dic[f"aims{side}"].vertices():
        vertex['p_value'] = 0.0  
        vertex_index[vertex.get('name')] = vertex

    for _, row in res.iterrows():
        name = row.vertex_name
        val = row[param]
        if name in vertex_index:
            vertex = vertex_index[name]

            vertex['p_value'] = val
                                                
                    # if row.p < -np.log10(0.05/56):
                    #     vertex['p_value'] = 0.
                    # elif vertex['p_value'] != 0.: 
                    #     vertex['p_value'] = min(vertex['p_value'],
                    #                             row.p)
                    # else:
                    #     vertex['p_value'] = row.p
    
    dic[f"ana{side}"] = a.toAObject(dic[f"aims{side}"])

    dic[f"ana{side}"].setColorMode(dic[f"ana{side}"].PropertyMap)
    dic[f"ana{side}"].setColorProperty('p_value')

    # pal = a.createPalette('VR-palette')
    # # pal.header()['palette_gradients'] = '0;0;0;0;0.456543;0;0.456543;1;1;1;1;1#0;0;0;0;0.465534;0;1;0;1;0;1;0;1;0#0;1;0;1;0.01998;1;0.102897;1;0.293706;1;0.471528;1;0.471528;0;1;0;1;0#0.182817;0.846154;0.386613;0.505495;0.453546;0;0.453546;0;0.484515;0;0.502498;0.0934066;0.523477;0.21978;0.716284;0.67033;1;1'
    # pal.header()['palette_gradients'] = '0;0;0.465714;0;0.471429;1;0.768571;1;1;1#0.00142857;0;0.0757143;0;0.124286;0.0384615;0.244286;0.392308;0.385714;0.815385;0.467143;1;0.547143;0.869231;0.62;0.707692;0.661429;0.607692;0.698571;0.553846;0.755714;0.423077;0.831429;0.276923;1;0#0;1;0.462857;1;0.462857;0.0230769;0.462857;0.0153846;0.462857;0;1;0#0.468571;1;1;1'
    # build_gradient(pal)

    #  # fix the palette to the full shift_x range
    # dic[f"ana{side}"].setPalette(
    #     "VR-palette",
    #     minVal=min_shift,
    #     maxVal=max_shift,
    #     absoluteMode=False
    # )

    dic[f"ana{side}"].notifyObservers()


    
                
def visualize_whole_hemisphere(view_quaternion, side, i):
    global block
    global dic
    global min_shift, max_shift
    threshold = max(abs(min_shift), abs(max_shift))
    print(min_shift, max_shift)
    
    try:
        block
    except NameError:
        block = a.createWindowsBlock(4)

    dic[f"win{i}"] = a.createWindow('3D',
                                    block=block,
                                    no_decoration=True,
                                    options={'hidden': 1})
    dic[f"win{i}"].addObjects(dic[f"ana{side}"])

    # pal = a.createPalette('VR-palette')
    # pal.header()['palette_gradients'] = '0;0;0;0;0.456543;0;0.456543;1;1;1;1;1#0;0;0;0;0.465534;0;1;0;1;0;1;0;1;0#0;1;0;1;0.01998;1;0.102897;1;0.293706;1;0.471528;1;0.471528;0;1;0;1;0#0.182817;0.846154;0.386613;0.505495;0.453546;0;0.453546;0;0.484515;0;0.502498;0.0934066;0.523477;0.21978;0.716284;0.67033;1;1'
    # pal.header()['palette_gradients'] = '0;0;0.465714;0;0.471429;1;0.768571;1;1;1#0.00142857;0;0.0757143;0;0.124286;0.0384615;0.244286;0.392308;0.385714;0.815385;0.467143;1;0.547143;0.869231;0.62;0.707692;0.661429;0.607692;0.698571;0.553846;0.755714;0.423077;0.831429;0.276923;1;0#0;1;0.462857;1;0.462857;0.0230769;0.462857;0.0153846;0.462857;0;1;0#0.468571;1;1;1'
    # build_gradient(pal)

     # fix the palette to the full shift_x range
    dic[f"ana{side}"].setPalette(
        "bwr",
        minVal=-threshold,
        maxVal=threshold,
        absoluteMode=True
    )

   
    # dic[f"ana{side}"].setPalette("green_yellow_red",
    #                           minVal=min_shift, maxVal=30,
    #                           absoluteMode=True)
    
    dic[f"win{i}"].camera(view_quaternion=view_quaternion)

middle_view = [0.5, -0.5, -0.5, 0.5]
side_view = [0.5, 0.5, 0.5, 0.5]
bottom_view = [0, -1, 0, 0]
top_view = [0, 0, 0, -1]
 
def visualize_whole(res, side, start):
    set_color_property(res, side)
    visualize_whole_hemisphere(middle_view if side == "L" else side_view, side, start+0)
    visualize_whole_hemisphere(top_view, side, start+1)
    visualize_whole_hemisphere(bottom_view, side, start+2)
    visualize_whole_hemisphere(side_view if side == "L" else middle_view, side, start+3)


In [18]:
dic = {}

visualize_whole(df_left, "L", 0)
visualize_whole(df_right, "R", 4)

min_shift: -5.82009158686731, max_shift: 6.442103452975935
Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60
-5.82009158686731 6.442103452975935
-5.82009158686731 6.442103452975935
-5.82009158686731 6.442103452975935
-5.82009158686731 6.442103452975935
min_shift: -11.200693703703706, max_shift: 6.9488707076197365
Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60
-11.200693703703706 6.9488707076197365
-11.200693703703706 6.9488707076197365
-11.200693703703706 6.9488707076197365
-11.200693703703706 6.9488707076197365


Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 8
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.


: 